## imports

In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2025-05-23 16:09:36.090903: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-23 16:09:36.206974: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-23 16:09:40.046706: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

In [2]:
import pickle
import sys
from tqdm import tqdm
sys.path.append('../')
sys.path.append('../../../../')

## constants

In [3]:
data_for_training_dir_path = source_dataset_dir_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/data_for_training'
source_dataset_dir_path = f'{data_for_training_dir_path}/03_04_with_pesto'
target_dataset_dir_path = f'{data_for_training_dir_path}/03_04_with_pesto_and_coord'

MAX_NUMBER_OF_COMPONENTS = 10

## utils

In [4]:
# copied from data_development_utils.py
# modified a bit
# TODO: the main risk here: sorted might change order from original creation 

def extract_protein_data(proteins, max_number_of_components):
    residue_indexes = []
    for protein in proteins:
        # Sort components by average_ubiq in descending order and take the top 10
        top_components = sorted(protein.connected_components_tuples, key=lambda x: x[1]['average_scanNet_ubiq'], reverse=True)[
                         :max_number_of_components]
        residue_indexes.append([component[2] for component in top_components])
    return residue_indexes

## load data from source

for fold_index in range(5) <br>
for set_name in ['train', 'validation', 'test'] <br>

Currently folds_training_dicts[fold_index] of set_name, contains:
- sizes (will be broadcasted)
- num_patches (will be broadcasted)
- components (the features matrix, of shape [protein_amount, 10, 9] -> 10=patches_amount, 9=2 for scannet, 5 for pesto, 1 for plddt, 1 for size)
- uniprots
- labels

We want to add one more matrix (again, per fold_index, per set_name). <br>
This matrix will be of shape [protein_amount, 10, 3] -> x, y, z coordiantes. <br>
x, y, z coordinates will be the average of the ca coords in the patch. <br>
the new key will be:

- coordinates

In [5]:
with open(f'{source_dataset_dir_path}/folds_training_dicts.pkl', 'rb') as f:
    folds_training_dicts = pickle.load(f)

2025-05-23 16:09:48.739723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# TODO: itterate in for loop
fold_index = 0
fold = folds_training_dicts[fold_index]
[x for x in fold.keys() if 'train' in x]

['sizes_train',
 'components_train',
 'num_patches_train',
 'uniprots_train',
 'labels_train']

In [7]:
with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/patches_dicts/03_04_with_pesto/merged_protein_objects.pkl', 'rb') as f:
# with open('/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/patches_dicts/03_04_with_pesto/proteinObjectsWithEvoluion0', 'rb') as f:
    merged_protein_objects = pickle.load(f)

/home/iscb/wolfson/doririmon/anaconda3/envs/ubinet-gpu/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [8]:
set_name = 'train'  # TODO: iterate in for loop
uniprots = set(fold[f'uniprots_{set_name}'])
len(uniprots), list(uniprots)[0]

(76632, 'Q9N3I3')

In [9]:
fold[f'components_{set_name}'].shape

TensorShape([76632, 10, 9])

In [10]:
data_set_protein_objects = {k: v for k, v in merged_protein_objects.items() if k in uniprots}
data_set_res_indexes = {k: extract_protein_data([v], MAX_NUMBER_OF_COMPONENTS)[0] for k, v in data_set_protein_objects.items()}
len(data_set_protein_objects), len(data_set_res_indexes)

(76632, 76632)

## add coord

In [11]:
# TODO
# we have the residue indexes in data_set_res_indexes
# now we just need to fetch the ca coord of each residue, then average accross the inner lists 

In [23]:
# lets try to get the ca coord of a single residue, from a protein object:
# TODO: this should iterate

data_set_coordinates = dict()


for uniprot_id, protein_object in tqdm(data_set_protein_objects.items()):
    structure = protein_object.get_structure()
    model = structure.child_list[0]
    assert len(model) == 1
    protein_indexes = data_set_res_indexes[uniprot_id]
    
    for patch_indexes in protein_indexes:
        for residue_index in patch_indexes:
            for chain in model:
                coord = chain.child_list[residue_index]['CA'].coord  # TODO: is this logic correct?


  0%|          | 2/76632 [00:00<1:22:10, 15.54it/s]

/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A022RQB5.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A022S055.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A059D4L0.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A061GXI7.pdb


  0%|          | 6/76632 [00:00<1:22:09, 15.54it/s]

/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A061H0D1.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A067FAT6.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A067FUV5.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A072UEZ0.pdb


  0%|          | 10/76632 [00:00<1:19:46, 16.01it/s]

...

  0%|          | 22/76632 [00:01<1:21:27, 15.68it/s]

/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A139WLH6.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A1D6EJ37.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A1D6IP61.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A1D6IQP4.pdb


  0%|          | 25/76632 [00:01<1:23:11, 15.35it/s]

/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A1D6JZI3.pdb
/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/datasets/patch_to_score/sources/GO/E1/A0A1D6QG29.pdb


KeyboardInterrupt: 

## save data to target